# 01 — Data preparation (ESS + Eurostat)

This notebook:
1. Loads ESS Round 11 microdata (CSV) from `data/raw/`
2. Cleans special missing codes and constructs **trust_index**
3. Loads Eurostat TSVs for **Gini** and **GDP per capita (PPS)** from `data/raw/`
4. Merges micro + macro data
5. Saves processed outputs to `data/processed/`

> **Important:** Do not upload raw ESS microdata to a public GitHub repo (license). Keep `data/raw/` local.


In [ ]:
# Imports
import re
from pathlib import Path

import numpy as np
import pandas as pd


In [ ]:
# Project paths (relative to repo root)


ROOT = Path().resolve().parent  # notebooks/ -> project root
DATA_RAW = ROOT / "data" / "raw"
DATA_PROCESSED = ROOT / "data" / "processed"
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

ESS_FILE  = DATA_RAW / "ees_round_11.csv"   # <-- rename if your file differs
GINI_FILE = DATA_RAW / "gini_path.tsv"
GDP_FILE  = DATA_RAW / "gdp_path.tsv"

YEAR = "2023"

for p in [ESS_FILE, GINI_FILE, GDP_FILE]:
    print("Expecting:", p)


In [ ]:
# 1) Load ESS microdata
# If your ESS file is large, you can optionally select columns with usecols=[...]
df = pd.read_csv(ESS_FILE)

print("ESS shape:", df.shape)
df.head()


In [ ]:
# 2) Construct trust index
trust_vars = ["ppltrst", "pplfair", "pplhlp"]

# ESS special missing codes -> NaN (common: 77, 88, 99)
for v in trust_vars:
    df[v] = df[v].replace([77, 88, 99], np.nan)

df["trust_index"] = df[trust_vars].mean(axis=1)

# Keep core variables needed for analysis
keep_cols = ["trust_index", "cntry", "agea", "gndr", "eisced", "hincfel"]
df_clean = df[keep_cols].copy()

# Drop missing trust/country
df_clean = df_clean.dropna(subset=["trust_index", "cntry"])

print("df_clean shape:", df_clean.shape)
df_clean.head()


In [ ]:
# 3) Load Eurostat TSVs (Gini + GDP)
gini_raw = pd.read_csv(GINI_FILE, sep="\t")
gdp_raw  = pd.read_csv(GDP_FILE,  sep="\t")

# Clean column names
gini_raw.columns = [c.strip() for c in gini_raw.columns]
gdp_raw.columns  = [c.strip() for c in gdp_raw.columns]

print("Gini raw columns:", list(gini_raw.columns)[:8], "...")
print("GDP  raw columns:", list(gdp_raw.columns)[:8], "...")


In [ ]:
# 4) Extract country codes (geo) from Eurostat dimension column
# Eurostat TSV first column often looks like: 'unit,indic_il,geo\time'
# Values may contain '... , AT' etc. We'll take the last comma-separated token.
gini_raw["geo"] = (
    gini_raw.iloc[:, 0].astype(str).str.split(",", expand=True).iloc[:, -1].str.strip()
)
gdp_raw["geo"] = (
    gdp_raw.iloc[:, 0].astype(str).str.split(",", expand=True).iloc[:, -1].str.strip()
)

def to_num(x):
    """Convert Eurostat cell values to float; ':' or missing -> NaN."""
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if s.startswith(":"):
        return np.nan
    m = re.search(r"[-+]?\d+(\.\d+)?", s)
    return float(m.group(0)) if m else np.nan

gini = gini_raw[["geo", YEAR]].rename(columns={YEAR: "gini"}).copy()
gdp  = gdp_raw[["geo", YEAR]].rename(columns={YEAR: "gdp_pps"}).copy()

gini["gini"] = gini["gini"].apply(to_num)
gdp["gdp_pps"] = gdp["gdp_pps"].apply(to_num)

macro = pd.merge(gini, gdp, on="geo", how="inner").dropna(subset=["gini", "gdp_pps"])

print("macro shape:", macro.shape)
macro.head()


In [ ]:
# 5) Merge micro + macro
df_merged = pd.merge(df_clean, macro, left_on="cntry", right_on="geo", how="inner")

print("Merged shape:", df_merged.shape)
print("Countries:", df_merged["cntry"].nunique())
df_merged.head()


In [ ]:
# 6) Save processed outputs
merged_out = DATA_PROCESSED / "ess_macro_merged.csv"
df_merged.to_csv(merged_out, index=False)
print("Saved:", merged_out)

# Country-level average trust (useful for country-level regressions & plots)
country_level = df_merged.groupby("cntry").agg({
    "trust_index": "mean",
    "gini": "first",
    "gdp_pps": "first"
}).reset_index()

country_out = DATA_PROCESSED / "country_trust_summary.csv"
country_level.to_csv(country_out, index=False)
print("Saved:", country_out)

country_level.head()
